# 스타벅스 매장 데이터와 지도


* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

# 스타벅스 매장찾기
http://www.istarbucks.co.kr

* STORE >> 매장 찾기
* 지역검색 >> 서울 >> 전체

In [6]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
# %config InlineBackend.figure_format = 'retina'    # retina(망막) display: 망막으로 구분할 수 없다는 뜻의 애플 용어

In [7]:
data = {
    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'126.97865',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)

r.text[:1000] # 수신된 데이터의 앞부분만 확인

'\n<!DOCTYPE html>\n<html>\n<head lang="ko">\n\t\r\n\r\n\n\n\n\n\n\n\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta charset="utf-8">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">\r\n<meta property="og:type"\t\tcontent="website">\r\n<meta property="og:title"\t\tcontent="Starbucks">\r\n<meta property="og:url"\t\t\tcontent="https://www.starbucks.co.kr/">\r\n<meta property="og:image"\t\tcontent="https://image.istarbucks.co.kr/common/img/kakaotalk.png">\r\n<meta property="og:description" content="Starbucks">\r\n\r\n<title id="titleJoin">Starbucks Coffee Korea</title>\r\n<link rel="shortcut icon" href="https://image.istarbucks.co.kr/common/img/common/favicon.ico?v=200828" type="image/ico"> <!-- 20200827 파비콘 교체 및 CDN 변수처리 -->\r\n<link href="/common/css/reset.css" rel="stylesheet">\r\n<link href="/common/css/style.css?v=200130" rel="stylesheet">\r\n<link href="/common/css/jquery.bxslider.css"

데이터는 JSON으로 수신되며, 형태는 다음과 같다.

```json
{
  "list": [
      {
            // ... 중략 ...
            "s_code": "1311",
            "s_name": "방화DT",
            "tel": "02-2664-3480",
            "fax": "02-2664-3481",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 방화동 293-4",
            "park_info": null,
            "new_state": null,
            "theme_state": "T17@T16@T09@T20@T01@T05@T08@T04",
            // ... 중략 ...
            "lat": "37.574339",
            "lot": "126.816415",
            "t22": 0
        },
        {
            // ... 중략 ...
            "s_code": "1267",
            "s_name": "마곡나루역",
            "tel": "02-3662-3504",
            "fax": "02-3662-3505",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 마곡동 759-3 보타닉파크타워Ⅰ105,203,204호",
            "park_info": null,
            "new_state": null,
            "theme_state": "T08@T05@T04@T17@T16@P80@T20",
            // ... 중략 ...
            "lat": "37.56813",
            "lot": "126.82614",
            "t22": 0
        },
    ]
 }
```

# JSON to DataFrame

json_normalize() 를 사용하여 JSON 데이터를 DataFrame로 전환 

In [8]:
jo = json.loads(r.text)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
# jo

In [ ]:
df = json_normalize(jo, 'list')

In [ ]:
# df

In [ ]:
# 행(row)수, 서울 457개 매장
len(df) 

In [ ]:
# 컬럼수 111개
df.columns

In [ ]:
# 주요한 컬럼 몇 가지 선택
               
df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df.head(20)

# 위도 경도 데이터 타입 변환

In [ ]:
df.dtypes

### lat, lot타입으로 전환 (str→float) 

In [ ]:
df['lat'] = df['lat'].astype(float)
df['lot'] = df['lot'].astype(float)

In [ ]:
df.dtypes

In [ ]:
df.head()

### 특정 지점

In [ ]:
df[df['s_name'] == '시청'] # 시청점

스타벅스 시청점의 위도(lat)와 경도(lot)는 각각  37.56629, 126.979808

# folium
https://github.com/python-visualization/folium

파이썬 지리정보 시각화 모듈  (문서: https://folium.readthedocs.io )

In [ ]:
!pip install folium

In [ ]:
import folium

# 서울 시청 
map_osm = folium.Map(location=(37.56629, 126.979808))
map_osm

지도를 상하좌우로 움직이거나 확대/축소 할 수 있다

In [ ]:
# 확대 지정 (zoom_start)
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm

In [ ]:
# 지도 tiles 지정
# map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17, tiles='Stamen Toner')
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm

In [ ]:
시청_좌표=(37.56629, 126.979808)
 
map_osm = folium.Map(location=시청_좌표, zoom_start=17)
folium.Marker(시청_좌표, popup='시청').add_to(map_osm)
map_osm

# 서울 스타벅스 전 지점

In [ ]:
map_osm = folium.Map(location=시청_좌표, zoom_start=11)

for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

# 제주도 스타벅스 매장 지도

In [ ]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

한라산_좌표 = (33.361666, 126.529166)

data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'16', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

map_osm = folium.Map(location=(33.361666, 126.529166), zoom_start=10)
for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

In [ ]:
df[['s_name', 'gugun_name', 'doro_address', 'tel']]

# 요약
* requests.post() 데이터 가져오기
* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

### 2018 FinanceData.KR [fb.com/financedata](fb.com/financedata)